# Estimation de la couche finale d'Akita sur les données de neurones

In [1]:
import json
import numpy as np
import pandas as pd
import sklearn
import statsmodels.api as sm

In [2]:
import h5py
import cooler

In [3]:
import matplotlib.pyplot as plt

In [4]:
predpath = "/home/bureau/projects/def-bureau/bureau/ran-donnees/PredictNeuronHi-C/akita_pred_sans_final/"
predfile = predpath + "preds.h5"

In [5]:
pred = h5py.File(predfile, 'r')
pred

<HDF5 file "preds.h5" (mode r)>

In [6]:
pred['preds'].shape

(7617, 99681, 48)

In [7]:
targetfile = "/home/bureau/projects/def-bureau/bureau/distiller/iPSC/data/1m/seqs_cov/0.h5"

In [8]:
targets = h5py.File(targetfile, 'r')
targets

<HDF5 file "0.h5" (mode r)>

In [9]:
# Les données d'élaboration sont les 7617 premières
train_targets = targets['targets'][:7617,]
train_targets.shape

(7617, 99681)

In [11]:
pred_mat = np.resize(pred['preds'],[pred['preds'].shape[0]*pred['preds'].shape[1],pred['preds'].shape[2]])
pred_mat.shape

(759270177, 48)

In [12]:
final_mod = sm.OLS(train_targets,pred_mat)

MemoryError: Unable to allocate 277. GiB for an array with shape (759270177, 49) and data type float64